# Dilepton analysis  


This is an example of a simple dilepton analysis, quite similar to the notebook called "Dilepton_analysis_noData.ipynb", but with some differences. The most obvious difference is that we here also include real data. This example also has a slightly more advanced event selection.  

**Notice:** This is *only an example* on how to do this. Feel free to be creative, and to find better and/or more elegant ways of doing the various steps! 

In [1]:
%jsroot on
#include <TMath.h>
#include <TLorentzVector.h>
#include <TChain.h>
#include <iostream>
#include <vector>
#include <map>
#include <TH1.h>
#include <TH1F.h>
#include <sstream>      // std::istringstream
#include <string>       // std::string

#include <fstream>
using namespace std;

## 1. Reading the dataset

In [ ]:
TChain *background = new TChain("mini");
TChain *data = new TChain("mini"); 

A list of all the background samples and their IDs can be found in **Background_samples.txt**. We read that list, and add all the samples to the TChain. We also (for later convenience) make a vector containing the dataset IDs. 

In [ ]:
TString sample; 
TString path; 
vector<Int_t> dataset_IDs;
Int_t DSID;
TString name;
TString cat;

In [ ]:
ifstream infile("../Input/Background_samples.txt");
ifstream infofile("../Input/Infofile.txt");

In [ ]:
vector<TString> Backgrounds; 
Backgrounds = {"Higgs","Diboson", "Wjets", "topX", "singleTop", "ttbar", "Zjets"};

In [ ]:
infofile.clear();
infofile.seekg(0, ios::beg);  // Start at the beginning of the file
std::map<TString,TString> bkg_dsid_toplot;
while(infofile >> name >> cat >> DSID){
    bkg_dsid_toplot[DSID] = cat;
}
infofile.close()

In [ ]:
infile.clear();
infile.seekg(0, ios::beg);  // Start at the beginning of the file
background->Reset(); // Reset the TChain (if necessary)  

while(infile >> sample >> DSID){
    //std::cout<<"checking "<<DSID<<std::endl;
    if ( bkg_dsid_toplot.find(DSID) == bkg_dsid_toplot.end() ){
        continue;
        std::cout<<"Skipping "<<DSID<<std::endl;
    }
    //printf("Checking %s \n",sample.Data());
    path = "/ATLAS_opendata/2lep/MC/"+sample; // Specify path to the samples 
    background->Add(path);  
    dataset_IDs.push_back(DSID);
    //std::cout<<"Adding "<<path.Data()<<std::endl;
    //std::cout<<"Adding "<<sample<<std::endl;
}
/**
    //
    path = "/ATLAS_opendata/2lep/MC/"+sample; // Specify path to the samples 
    std::cout<<"Adding "<<sample<<std::endl;
    background->Add(path);  
    dataset_IDs.push_back(DSID);
}
*/

In [ ]:
background->GetEntries()

In [ ]:
data->Reset(); 
data->Add("/ATLAS_opendata/2lep/Data/*.root"); 
//data->Add("http://opendata.atlas.cern/release/samples/Data/DataMuons.root"); 
data->GetEntries()

In [ ]:
printf("Number of events data = %lld\n",data->GetEntries());
printf("Number of events bkg  = %lld\n",background->GetEntries());

Next we define the variables we want to include in the analysis, and link them to branches in the TTree. A few things to notice at this point: 
-  In this example we will only study events with two leptons, so the vectorial variables only need to be two dimensional. 
-  The variables are here given names corresponding to the branches in the TTree. This is not necessary, so if you want to give them other names you are free to do so. 
-  The variable called "channelNumber" is the same as we have called "dataset ID" above. These terms are used interchangeably. 

In [ ]:
// It is important to set the pointers to zero. If not you will get a segmentation violation when running
 unsigned int lep_n;
  vector<int> *lep_charge = 0;
  int channelNumber;
  vector<UInt_t> *lep_type = 0;
  vector<float> *lep_pt = 0;
  vector<float> *lep_E = 0;
  vector<float> *lep_phi = 0;
  vector<float> *lep_eta = 0;
  vector<float> *jet_MV2c10 = 0;
  vector<float> *lep_etcone20 = 0;
  vector<float> *lep_ptcone30 = 0;  
  float met_et;
  vector<bool> *lep_trigMatched = 0;
  bool trigE;
  bool trigM; 

In [ ]:
float scaleFactor_PILEUP; 
float scaleFactor_ELE;
float scaleFactor_MUON;
float scaleFactor_BTAG;
float scaleFactor_LepTRIGGER;
float mcWeight;

In [ ]:
 background->SetBranchAddress("lep_n",      &lep_n);
 
  background->SetBranchAddress("lep_charge", &lep_charge);
  background->SetBranchAddress("lep_type",   &lep_type);
  background->SetBranchAddress("lep_pt",     &lep_pt);
  background->SetBranchAddress("lep_eta",    &lep_eta);
  background->SetBranchAddress("lep_phi",    &lep_phi);
  background->SetBranchAddress("lep_E",      &lep_E);
  background->SetBranchAddress("met_et",     &met_et); 
  background->SetBranchAddress("channelNumber", &channelNumber);
  background->SetBranchAddress("mcWeight", &mcWeight); 
  background->SetBranchAddress("scaleFactor_PILEUP", &scaleFactor_PILEUP ); 
  background->SetBranchAddress("scaleFactor_ELE", &scaleFactor_ELE ); 
  background->SetBranchAddress("scaleFactor_MUON", &scaleFactor_MUON ); 
  background->SetBranchAddress("scaleFactor_BTAG", &scaleFactor_BTAG ); 
  background->SetBranchAddress("scaleFactor_LepTRIGGER", &scaleFactor_LepTRIGGER ); 
  background->SetBranchAddress("lep_ptcone30", &lep_ptcone30); 
  background->SetBranchAddress("lep_etcone20", &lep_etcone20); 
  background->SetBranchAddress("trigE", &trigE); 
  background->SetBranchAddress("trigM", &trigM); 
  background->SetBranchAddress("jet_MV2c10", &jet_MV2c10); 

  // For data 
  data->SetBranchAddress("lep_n",      &lep_n);
  data->SetBranchAddress("lep_charge", &lep_charge);
  data->SetBranchAddress("lep_type",   &lep_type);
  data->SetBranchAddress("lep_pt",     &lep_pt);
  data->SetBranchAddress("lep_eta",    &lep_eta);
  data->SetBranchAddress("lep_phi",    &lep_phi);
  data->SetBranchAddress("lep_E",      &lep_E);
  data->SetBranchAddress("met_et",     &met_et); 
  data->SetBranchAddress("channelNumber", &channelNumber);
  data->SetBranchAddress("trigE", &trigE); 
  data->SetBranchAddress("trigM", &trigM); 
  data->SetBranchAddress("lep_trigMatched", &lep_trigMatched); 
  data->SetBranchAddress("lep_ptcone30", &lep_ptcone30); 
  data->SetBranchAddress("lep_etcone20", &lep_etcone20); 
  data->SetBranchAddress("jet_MV2c10", &jet_MV2c10); 

## 2. Making (a lot of) histograms

Now that we have read our dataset we want to start analyzing the data. To do so we need to put the data into histograms. For reasons that will become clear later in the analysis we must (for each variable) make one histogram per dataset ID. (We have 31 background samples, so if we want to study 10 variables we have to make 310 histograms!) A very elegant way of dealing with all these histograms is by using [map](http://www.cplusplus.com/reference/map/map/)s (the C$++$ equivalent of Python dictionaries). Below we define one map for each variable. Here the *key values* are the dataset IDs, while the *mapped values* are the histograms.   

In [ ]:
map<Int_t, TH1*> hist_mll; 
map<Int_t, TH1*> hist_lep_pt; 
map<Int_t, TH1*> hist_met;

In [ ]:
for(const auto & i:dataset_IDs){
    hist_mll[i] = new TH1F(); 
    hist_lep_pt[i] = new TH1F(); 
    hist_met[i] = new TH1F();
}

In [ ]:
for(const auto & i:dataset_IDs){
    hist_mll[i]->SetNameTitle("hist_mll", "Invariant mass"); 
    hist_lep_pt[i]->SetNameTitle("hist_lep_pt", "Lepton pT"); 
    hist_met[i]->SetNameTitle("hist_met", "Missing ET");
    hist_mll[i]->SetBins(20,0,500); 
    hist_lep_pt[i]->SetBins(20,0,1000);
    hist_met[i]->SetBins(20,0,500); 
}

For data it is only necessary with one histogram for each variable: 

In [ ]:
TH1F *hist_mll_d = new TH1F(); 
TH1F *hist_lep_pt_d = new TH1F(); 
TH1F *hist_met_d = new TH1F(); 

In [ ]:
hist_mll_d->SetNameTitle("hist_mll", "Invariant mass"); 
hist_lep_pt_d->SetNameTitle("hist_lep_pt", "Lepton pT"); 
hist_met_d->SetNameTitle("hist_met", "Missing ET");
hist_mll_d->SetBins(20,0,500); 
hist_lep_pt_d->SetBins(20,0,1000);
hist_met_d->SetBins(20,0,500); 

### 2.1 Fill the histograms 
We can now loop over all events in our dataset, implement desired cuts, and fill the histograms we created above. In this example we choose only events containing exactly to same flavour leptons with opposite charge (i.e. $e^+e^-$ or $\mu^+\mu^-$). 
Before starting the loop we extract the total number of entries (events) in the TChain. We also make [TLorentzVector](https://root.cern.ch/doc/master/classTLorentzVector.html)s, which are very practical for handling the kinematics of the leptons, e.g. calculating the invariant mass of the two leptons. 

In [ ]:
TLorentzVector l1, l2, dileptons; 

In [ ]:
TChain *dataset = new TChain("mini"); 
int isData = 0; 
int nentries; 
Float_t W; 
int n_bjets; 

In [ ]:
// Reset histograms (in case you have filled them before) 
for(const auto & i:dataset_IDs){ 
    hist_mll[i]->Reset(); 
    hist_lep_pt[i]->Reset(); 
    hist_met[i]->Reset();
}

hist_mll_d->Reset(); 
hist_lep_pt_d->Reset(); 
hist_met_d->Reset(); 
std::cout<<"isData = "<<isData<<std::endl;

In [ ]:
// Loop through all events 
for(isData = 0; isData<2; isData++){
    
    std::cout<<"isData = "<<isData<<std::endl;
/**
    if(isData == 1){ 
        nentries = data->GetEntries();
        dataset = data; 
        std::cout << "Running over data..." << std::endl; 
    }
    else {
   */
   nentries = background->GetEntries();
    /** 
    dataset = background;      
        std::cout << "Running over background..." << std::endl; 
    }
    */
    
  for (int i = 0; i < nentries; i++){
 
      if( i%1000000 == 0 && i>0){ std::cout << i/1000000 << " million events processed" << std::endl;}
       
      //std::cout<<"i = "<<i<<std::endl;
       
      background->GetEntry(i); // We "pull out" the i'th entry in the chain. The variables are now 
      // available through the names we have given them. 
       /**
      
      //if(!( i%100000==0)){ continue; }

      // Data quality cuts: 

      // if(passGRL == 0){ continue; } 
      //if(hasGoodVertex == 0){ continue; }
      //if(trigM == 0 && trigE == 0){ continue; } 

      // Require "good leptons":
      if(lep_n != 2){ continue; }

      if( lep_pt->at(0)/1000.0 < 25 ){ continue; }
        
      if( lep_etcone20->at(0)/lep_pt->at(0) > 0.15 ){ continue; }
      if( lep_ptcone30->at(0)/lep_pt->at(0) > 0.15 ){ continue; }
      //if( !(lep_flag->at(0) & 512) ){ continue; }

      if( lep_pt->at(1)/1000.0 < 25 ){ continue; }
      if( lep_etcone20->at(1)/lep_pt->at(1) > 0.15 ){ continue; }
      if( lep_ptcone30->at(1)/lep_pt->at(1) > 0.15 ){ continue; }
      //if( !(lep_flag->at(1) & 512) ){ continue; }

      // Event selection: 

      // Cut #1: Require (exactly) 2 leptons
      if(lep_n != 2){ continue; }
      // Cut #2: Require opposite charge
      if(lep_charge->at(0) == lep_charge->at(1)){ continue; }
      // Cut #3: Require same flavour (2 electrons or 2 muons)
      if(lep_type->at(0) != lep_type->at(1)){ continue; }

      // Set Lorentz vectors: 
      l1.SetPtEtaPhiE(lep_pt->at(0)/1000., lep_eta->at(0), lep_phi->at(0), lep_E->at(0)/1000.);
      l2.SetPtEtaPhiE(lep_pt->at(1)/1000., lep_eta->at(1), lep_phi->at(1), lep_E->at(1)/1000.);
      // Variables are stored in the TTree with unit MeV, so we need to divide by 1000 
      // to get GeV, which is a more practical and commonly used unit. 

      dileptons = l1 + l2;       

*/
    }
  }

//}  
//std::cout << "Done!" << std::endl; 

In [ ]:
dataset->GetEntry(2)

We have now done the "heavy lifting" of an analysis, i.e. looping through all the events. Usually in such an analysis we create new ROOT files where we store the histograms we made above, and then analyse the output in a separate program/script. The advantage of doing this is that you can do the rest of the analysis in another language, e.g. Python, since we are done with part that requires the speed of C$++$. If you want to write ROOT files you can check out the [TFile](https://root.cern.ch/doc/master/classTFile.html) class reference. In this example we will however carry on in C$++$. But even if you only work in C++ it can be practical to write the histograms to file, so that you don't need to run the loop _every time_ you open the notebook.  

## 3. Scale and classify the histograms (MC only) 

Before we are ready to make plots we need to do some further processing of the histograms we made above. The information necessary for doing the two steps below is found in the file **Infofile.txt**.   
1. We need to **scale** the histograms to the right cross section and luminosity. Why? When making the MC samples a certain number of events is simulated, which will usually not correspond to the number of events in our data. The expected number of events from a certain kind of process is given by $N=\sigma L$, where $\sigma$ is the cross section and $L$ is the integrated luminosity. Therefore we need to scale each histogram by a scale factor <br> <br>
$$sf = \frac{N}{N_{MC}} = \frac{ \sigma L }{N_{MC}},$$ <br>  where $N_{MC}$ is the number of generated MC events.  <br> <br>
2. We also need to **classify** the background processes into different categories. This is necessary when we eventually want to make the characteristic colorful background plots you might have seen before.  

### 3.1 Make new histograms 
Maybe a bit depressingly we have to make a set of new histograms, this time corresponding to the different background categories, instead of the dataset IDs. Notice that these new histograms are made in a very similar way as above, i.e. with the same range and binning. 

In [2]:
map<TString, TH1*> H_mll; 
map<TString, TH1*> H_lep_pt; 
map<TString, TH1*> H_met;

In [3]:
vector<TString> Backgrounds; 

In [4]:
Backgrounds = {"Higgs","Diboson", "Wjets", "DY", "singleTop", "ttbar", "Zjets"}; 

In [5]:
for(const auto & i:Backgrounds){
    H_mll[i] = new TH1F(); 
    H_lep_pt[i] = new TH1F(); 
    H_met[i] = new TH1F(); 
}

In [6]:
for(const auto & i:Backgrounds){
    H_mll[i]->SetNameTitle("hist_mll", "Invariant mass"); 
    H_lep_pt[i]->SetNameTitle("hist_lep_pt", "Lepton pT"); 
    H_met[i]->SetNameTitle("hist_met", "Missing ET");
    H_mll[i]->SetBins(20,0,500); 
    H_lep_pt[i]->SetBins(20,0,1000);
    H_met[i]->SetBins(20,0,500); 
}

### 3.2 Scale and add histograms 
Now we read our info file, scale all (old) histograms, and then add them to the new histograms we just defined.  

In [7]:
L = 1000.6; 

In [8]:
for(const auto & i:Backgrounds){
    H_mll[i]->Reset(); 
    H_lep_pt[i]->Reset(); 
    H_met[i]->Reset();
}

In [ ]:
std::string ifile;
ifile = "../Input/Files_base.txt";
ifstream input(ifile.c_str());
std::string line;
while(getline(input,line)){
    cout<<"line = "<<line<<endl;
    if (line.find("#") != line.npos )continue; // a # is a comment and not read
    std::string name, xsec, sumw, eff;
    istringstream linestream(line);
    getline(linestream, name, '|');
    getline(linestream, xsec, '|');
    getline(linestream, sumw, '|');
    getline(linestream, eff);
    if (DEBUG) cout << name << " " << xsec << " " << sumw << " " << eff << endl;
    float sumw_eff = atof(sumw.c_str()) * atof(eff.c_str());
    SF[name] = std::make_pair(atof(xsec.c_str()), sumw_eff);    
cout<<"name = "<<name<<endl;
}

//info.clear();
//info.seekg(0, ios::beg);  
//while(info >> process >> type >> dsid >> n_events >> red_eff >> sum_w >> x_sec){
    SF = x_sec*L/(sum_w*red_eff); 

    hist_mll[dsid]->Scale(SF); 
    hist_lep_pt[dsid]->Scale(SF); 
    hist_met[dsid]->Scale(SF); 
    
    H_mll[type]->Add(hist_mll[dsid]); 
    H_lep_pt[type]->Add(hist_lep_pt[dsid]); 
    H_met[type]->Add(hist_met[dsid]); 
    
#}

### 3.3 Color the histograms 
Make yet another map, this time containing the colors you want the backgrounds to have, and then set the colors of your histograms. Note that colors are defined by integers in ROOT. If you are not happy with the colors chosen below you can have look at the [TColor](https://root.cern.ch/doc/master/classTColor.html) class reference for more options. 

In [ ]:
map<TString, Int_t> colors; 

In [ ]:
colors["Diboson"] = kGreen; 
colors["Zjets"] = kYellow; 
colors["ttbar"] = kRed;
colors["singleTop"] = kBlue-7; 
colors["Wjets"] = kBlue+3; 
colors["DY"] = kOrange+1; 
colors["Higgs"] = kMagenta; 

In [ ]:
for(const auto h:Backgrounds){
    H_mll[h]->SetFillColor(colors[h]); 
    H_met[h]->SetFillColor(colors[h]);
    H_lep_pt[h]->SetFillColor(colors[h]);
    
    H_mll[h]->SetLineColor(colors[h]); 
    H_met[h]->SetLineColor(colors[h]);
    H_lep_pt[h]->SetLineColor(colors[h]);
}

## 4. Stack and plot the histograms

Finally we have arrived to the part where we can plot the results of all the work done above. For each variable we need to stack the backgrounds on top of each other, which is done by using the [THStack](https://root.cern.ch/doc/master/classTHStack.html) class. In the example below we do this for two variables; invariant mass and missing $E_T$.   

In [ ]:
THStack *stack_mll = new THStack("Invariant mass", "");
THStack *stack_met = new THStack("Missing ET", ""); 
THStack *stack_lep_pt = new THStack("Lepton pT", ""); 

In [ ]:
for(const auto h:Backgrounds){
    stack_mll->RecursiveRemove(H_mll[h]); // Remove previously stacked histograms  
    stack_met->RecursiveRemove(H_met[h]);
    stack_lep_pt->RecursiveRemove(H_lep_pt[h]);
    stack_mll->Add(H_mll[h]); 
    stack_met->Add(H_met[h]);
    stack_lep_pt->Add(H_lep_pt[h]); 
}    

Now we make a legend (see [TLegend](https://root.cern.ch/doc/master/classTLegend.html)), and add  the different backgrounds. Next we make a canvas (see [TCanvas](https://root.cern.ch/doc/master/classTCanvas.html)), which is allways necessary when we want to make a plot. Then you draw the stack and the legend, and display them by drawing the canvas. We can also specify axis labels and a bunch of other stuff. 

In [ ]:
gStyle->SetLegendBorderSize(0); // Remove (default) border around legend 
TLegend *leg = new TLegend(0.65, 0.60, 0.9, 0.85); 

In [ ]:
leg->Clear();
for(const auto i:Backgrounds){
    leg->AddEntry(H_mll[i], i, "f");  // Add your histograms to the legend
} 
leg->AddEntry(hist_mll_d, "Data", "lep"); 

In [ ]:
TCanvas *C = new TCanvas("c", "c", 600, 600);

In [ ]:
gPad->SetLogy(); // Set logarithmic y-axis

In [ ]:
hist_mll_d->SetLineColor(kBlack); 
hist_mll_d->SetMarkerStyle(kFullCircle); 
hist_mll_d->SetMarkerColor(kBlack); 

In [ ]:
stack_mll->Draw("hist"); 
stack_mll->SetMaximum(1E6); 
stack_mll->GetYaxis()->SetTitle("# events");
stack_mll->GetYaxis()->SetTitleOffset(1.3); 
stack_mll->GetXaxis()->SetTitle("m_{ll} (GeV)");
stack_mll->GetXaxis()->SetTitleOffset(1.3);
hist_mll_d->Draw("same E"); 
leg->Draw();
C->Draw();

In [ ]:
hist_met_d->SetLineColor(kBlack); 
hist_met_d->SetMarkerStyle(kFullCircle); 
hist_met_d->SetMarkerColor(kBlack); 

In [ ]:
stack_met->Draw("hist"); 
stack_met->SetMaximum(1E6); 
stack_met->SetMinimum(1E-2); 
stack_met->GetYaxis()->SetTitle("# events");
stack_met->GetYaxis()->SetTitleOffset(1.3); 
stack_met->GetXaxis()->SetTitle("E_{T}^{miss} (GeV)");
stack_met->GetXaxis()->SetTitleOffset(1.3);
hist_met_d->Draw("same e"); 
leg->Draw();
C->Draw(); 

In [ ]:
hist_lep_pt_d->SetLineColor(kBlack); 
hist_lep_pt_d->SetMarkerStyle(kFullCircle); 
hist_lep_pt_d->SetMarkerColor(kBlack); 

In [ ]:
stack_lep_pt->Draw("hist"); 
stack_lep_pt->SetMaximum(1E6); 
stack_lep_pt->GetYaxis()->SetTitle("# events");
stack_lep_pt->GetYaxis()->SetTitleOffset(1.3); 
stack_lep_pt->GetXaxis()->SetTitle("p_{T} (GeV)");
stack_lep_pt->GetXaxis()->SetTitleOffset(1.3);
hist_lep_pt_d->Draw("same e"); 
leg->Draw();
C->Draw(); 